# Section 1: Behavior

## Demographics

In [ ]:
from pandas import read_csv

## Load demographics data.
df = read_csv('demographics.csv')

## Calculate average age.
print df.Age.mean(), df.Age.std()

## Tabulate genders before rejection.
print df.Gender.value_counts()
print df[~df.Exlude].Gender.value_counts()

## Assemble Behavior Data

In [ ]:
import os
import numpy as np
from pandas import concat, read_csv
csv_dir = 'behavior'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

modality = 'mri'
scanner_fix = 2.95

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Load and assemble data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Define subject list.
df = read_csv('demographics.csv')
subjects = df.loc[~df.Exlude,'Subject']

df = []
for subject in subjects:
    
    f = os.path.join(csv_dir, '%s_arc_%s_ser-1' %(subject,modality))
    csv = read_csv(os.path.join(csv_dir,f))

    ## Add subject. Remove missing trials.
    csv['Subject'] = subject

    ## Append.
    df.append(csv)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## Merge and preprocess.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
## Merge and crop.
columns = ['Subject', 'Trial', 'RiskType', 'RewardType', 'Reward', 'ResponseType', 
           'ResponseTime','RiskOnset','ShockOnset']
df = concat(df)[columns].reset_index(drop=True)

## Replace missing responses.
df.loc[df.ResponseType==99,'ResponseType'] = np.nan

## Due to code-scanner issues, we will impute all reaction times greater than 2.95s.
if scanner_fix: df.loc[df.ResponseTime>scanner_fix, 'ResponseTime'] = np.nan

## Save.
df.to_csv('stan_results/arc_%s_FINAL.csv' %modality, index=False)
print 'Done.'

## Bayesian Modeling

### Understanding the Gamma Distribution

In [ ]:
import numpy as np
import pylab as plt
from ipywidgets import interact
from scipy.stats import gamma
%matplotlib inline

def visualize(mu, k):
    scale = mu / float(k)
    x = np.linspace(0,3,1e3)
    y = gamma.pdf(x, k, scale=scale)
    plt.plot(x,y)
    plt.ylim(0,5)
    plt.tight_layout()
    plt.show()
    
interact(visualize, mu=(0,3,0.1), k=(0,100,1))

x = np.linspace(0,1e2,1e3)
y = gamma.pdf(x, 1, scale=1/0.05)
plt.plot(x,y)

### Run Models with Stan
See [here](https://github.com/stan-dev/stan/wiki/Prior-Choice-Recommendations) for discussion of choice of logistic priors. Namely:
>Assuming that nonbinary variables have been scaled to have mean 0 and standard deviation 0.5, [Gelman et al (2008)](https://arxiv.org/pdf/0901.4011.pdf) recommended *student_t(1,0,2.5)*, i.e. Cauchy distribution. Later it has been observed that this has too thick tails, so that in cases where data is not informative (e.g. in case of separation) the sampling from the posterior is challenging (see e.g. [Ghosh et al, 2015](http://arxiv.org/abs/1507.07170)). Thus Student's t distribution with higher degrees of freedom is recommended. There is not yet conclusive results what specific value should be recommended, and thus the current recommendation is to choose 3<nu<7. 

>Normal distribution is not recommended as a weakly informative prior, because it is not robust (see [O'Hagan (1979)](https://www.jstor.org/stable/2985064)). Normal distribution would be fine as an informative prior.



In [ ]:
import cPickle, os, pystan
import numpy as np
from pandas import DataFrame, read_csv, get_dummies
def zscore(arr): return (arr - np.nanmean(arr)) / np.nanstd(arr) 

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## Specify parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## I/O parameters.
file_name = 'arc_mri_FINAL.csv'
model_name = 'arc_non-hierarchical_FINAL'
out_name = 'arc_non-hierarchical_add_FINAL2'
interactions = False

## Model parameters
nu = 5

## Sampling parameters.
chains = 4
samples = 2000
thin = 4
seed = 47404
n_jobs = 2

print('n_samples: %s' %(chains * samples / 2 / thin))

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## Assemble data for model.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Load data.
df = read_csv('stan_results/%s' %file_name)
df = df[~np.isnan(df.ResponseType)].reset_index(drop=True)

## Make subject index.
subjects, ix = np.unique(df.Subject, return_inverse=True)        
ix += 1

## Make missing data index.
mi = df.ResponseTime.isnull().astype(int)
mi *= np.cumsum(mi)

## Assemble indepedent variables.
_, med_risk, high_risk = get_dummies(df.RiskType).as_matrix().T 
risk = np.vstack([med_risk, high_risk])
intercept = np.ones_like(med_risk)
reward = df.Reward.as_matrix()                      

if interactions: X = np.vstack([intercept,risk,reward,risk*reward]).T 
else: X = np.vstack([intercept,risk,reward]).T  

## Assemble depedent variables.
N = df.ResponseType.as_matrix().astype(int)
Z = df.ResponseTime.as_matrix()
Z[np.isnan(Z)] = 99

## Z-score variables.
zX = X.copy()
if not interactions: 
    zX[:,-1] = zscore(X[:,-1])
else:
    zX[:,3] = zscore(zX[:,3])
    zX[:,4] = zX[:,1] * zX[:,3]
    zX[:,5] = zX[:,2] * zX[:,3]

## Assemble metadata.
n_obs, n_pred = X.shape
n_subj = subjects.shape[0]
n_miss = len(mi.nonzero()[0])

## Assemble data.
data = dict(n_obs=n_obs, n_pred=n_pred, n_subj=n_subj, n_miss=n_miss, ix=ix, mi=mi, X=zX, N=N, Z=Z)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## Perform Bayesian modeling.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

print 'Running Stan.',
f = 'stan_models/%s.txt' %model_name
fit = pystan.stan(file=f, data=data, chains=chains, iter=samples, thin=thin,
                  seed=seed, n_jobs=n_jobs)
print 'Finished.'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## Save summary file.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

summary = fit.summary()
summary = DataFrame(summary['summary'], columns=summary['summary_colnames'], index=summary['summary_rownames'])
f = 'stan_results/%s.csv' %out_name
summary.to_csv(f)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## Extract Results.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

results = fit.extract()

## Append data to results.
results['Subjects'] = subjects
results['X'] = X
results['zX'] = zX
results['N'] = N
results['Z'] = Z
results['ix'] = ix
results['RiskOnset'] = df.RiskOnset.as_matrix()

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## Save results.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
print 'Saving data.'

## Save all data.
f = 'stan_results/%s.pickle' %out_name
with open(f, 'wb') as f: cPickle.dump(results, f)
    
## Save log-likelihood for R.
np.savetxt('stan_results/%s_loglik.txt' %out_name, np.log(results[u'PointPosteriors']))

print 'Done.'

### Posterior Predictive Checks

In [ ]:
import cPickle, os
import numpy as np
import pylab as plt
from pandas import DataFrame

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

results_file = 'arc_non-hierarchical_add_FINAL2'
ncol = 4
decim = 4

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Load data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Open results file.
print 'Loading data.'
f = 'stan_results/%s.pickle' %results_file
with open(f, 'rb') as f: results = cPickle.load(f)

## Extract variables.
subjects = results['Subjects']
ix = results['ix'] - 1
n_subjects = ix.max() + 1
nrow = int(np.ceil(n_subjects / ncol))

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Evaluate fit to choice data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#   

print 'Evaluating choice data.'

def bernoulli(p): return np.random.binomial(1,p,size=1)
Bernoulli = np.vectorize(bernoulli)

## Compute true ratio of takes.
ratio_obs = np.array([results['N'][ix==i].mean() for i in range(n_subjects)])

## Generate simulations of take/no-take.
np.random.seed(47404)
sN = Bernoulli(results['theta'])

## Compute ratio take from simulations.
ratio_sim = np.array([sN[:,ix==i].mean(axis=1) for i in range(n_subjects)])

## Compute difference between observed and median of distribution.
ppc = ratio_obs - np.median(ratio_sim, axis=1)

## Compute group statistic.
rms_ppc = np.sqrt(np.power(ppc,2).mean())

## Initialize plots.
fig, axes = plt.subplots(nrow,ncol,figsize=(nrow*3,ncol*4),sharey=True)

for n, subject in enumerate(subjects):
    
    axes[n/ncol,n%ncol].hist(ratio_sim[n], bins=8, color='#7ec0ee')
    axes[n/ncol,n%ncol].vlines(ratio_obs[n], 0, 500, linewidth=2, linestyle='--')
    axes[n/ncol,n%ncol].set_title(subject.upper())
    x1, x2 = axes[n/ncol,n%ncol].get_xlim()
    xticks = np.linspace(x1,x2,3).round(2)
    axes[n/ncol,n%ncol].set_xticks(xticks)
    
plt.suptitle('PPC = %0.3f' %rms_ppc, y=0.99)
plt.tight_layout()
# plt.show()
plt.savefig('plots/ppc/%s_ppc_choice.png' %results_file)
plt.close('all')

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Evaluate fit to reaction time data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#  

print 'Evaluating reaction time data.'

## Extract and provide imputations data.
Z = results['Z'].copy()
Z[np.where(Z==99)] = results['Zm'].mean(axis=0)

## Initialize plots.
fig, axes = plt.subplots(nrow,ncol,figsize=(nrow*3,ncol*4))

for n, subject in enumerate(subjects):

    ## Extract info.
    a0 = results['a0'][:,n]
    if 'non-' in results_file: a1 = results['a1_mu']
    else: a1 = results['a1'][:,n]
    shape = results['shape'][:,n]
    ddb = results['ddb'][:,ix==n]

    ## Calculate gamma parameters.
    mu = a0 + (a1 * ddb.T) 
    scale = mu / shape

    ## Simulate reaction times.
    np.random.seed(47404)
    def gamma(shape,scale): return np.random.gamma(shape,scale,size=1)
    Gamma = np.vectorize(gamma)
    rt_sim = Gamma(shape,scale)

    ## Extract observed reaction times.
    rt_obs = Z[ix==n]

    ## Plot observed.
    density, bins = np.histogram(rt_obs, bins=5, density=True)
    x = bins[:-1] + np.diff(bins) 
    axes[n/ncol,n%ncol].plot(x,density,linewidth=3,color='#7ec0ee')
    axes[n/ncol,n%ncol].set_title(subject.upper())
    axes[n/ncol,n%ncol].set_xlim(0,5)

    ## Plot simulated.
    for arr in rt_sim.T[::decim]:
        density, bins = np.histogram(arr, bins=5, density=True)
        x = bins[:-1] + np.diff(bins) 
        axes[n/ncol,n%ncol].plot(x,density,color='k',alpha=0.01)

plt.tight_layout()
# plt.show()
plt.savefig('plots/ppc/%s_ppc_rt.png' %results_file)
plt.close('all')
print 'Done.'

### Model Comparison: Compute WAIC / CV-LOO
Performed in R. See other script.

### Assemble model outputs for fMRI Analysis

In [ ]:
import cPickle, os
import numpy as np
from pandas import DataFrame, read_csv

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

results_file = 'arc_hierarchical_add_FINAL2'
csv_file = 'arc_mri_FINAL'
out_file = 'arc_hierarchical_add_FINAL2_regressors'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Load data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Open results file.
print 'Loading data.'
f = 'stan_results/%s.pickle' %results_file
with open(f, 'rb') as f: results = cPickle.load(f)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Build dataframe.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

print 'Assembling dataframe.'

## Initialize.
df = dict()

## Add subject info.
ix = results['ix'] - 1
df['Subject'] = results['Subjects'][ix]

## Add regressors.
df['RiskType'] = np.where(results['X'][:,1]==1, 2, np.where(results['X'][:,2]==1,3,1))
df['Reward']   = results['X'][:,3]
df['theta']    = np.median(results['theta'], axis=0)
df['ddb']      = 0.25 - np.power(df['theta'] - 0.5, 2)

## Extract and provide imputations data.
Z = results['Z'].copy()
Z[np.where(Z==99)] = results['Zm'].mean(axis=0)
df['RT'] = Z
df['RiskOnset'] = results['RiskOnset']

## Assemble dataframe.
df = DataFrame(df)

## Merge with original file.
csv = read_csv('stan_results/%s.csv' %csv_file)
csv.drop('ResponseTime', axis=1, inplace=True)

df = df.merge(csv, how='outer', on=['Subject','RiskType','Reward','RiskOnset'])
df = df.sort_values(['Subject','Trial']).reset_index(drop=True)
df = df[['Subject','Trial','RiskType','ResponseType','ddb','RT','RiskOnset','ShockOnset']]

## Save.
print 'Saving dataframe.'
df.to_csv('stan_results/%s.csv' %out_file, index=False)

print 'Done.'

# Section 2: MRI Preprocessing

## Motion Correction

### Prepare White Matter Masks
Please see: /space/sophia/2/users/DARPA-Behavior/notebooks/bayes/decision_making/NN_bayes_2016/scripts/wm_masks.csh

### Extract and Store Grey/White Matter

In [ ]:
import os
import numpy as np
import nibabel as nib
from pandas import read_csv
from mne.filter import construct_iir_filter, filter_data
def demean(arr): return arr - arr.mean()

mri_dir = '/autofs/space/lilli_002/users/DARPA-ARC/'
subjects_dir = '/autofs/space/lilli_001/users/DARPA-Recons'
out_dir = '/space/sophia/2/users/DARPA-Behavior/notebooks/bayes/decision_making/NN_bayes_2016/motion'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

decim = 250
tr = 1.75
high_pass = 100 # seconds

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

info = read_csv('demographics.csv')
subjects = info.loc[~info.Exlude,'Subject'].as_matrix()

for subject in subjects:
    
    print subject,
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Load and prepare masks.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    brainmask = os.path.join(mri_dir, subject, 'arc_001', '001', 'masks', 'brain.nii.gz')
    brainmask = np.where( nib.load(brainmask).get_data(), 1, 0 ) # Binarize the mask

    wm = os.path.join(mri_dir, subject, 'arc_001', '001', 'masks', 'wm.mgz')
    wm = np.where( nib.load(wm).get_data(), 1, 0 ) # Binarize the mask

    gm = brainmask - wm
    
    ## Reduce to indices of interest.
    gm = np.vstack(np.where(gm))[:,::decim]
    wm = np.vstack(np.where(wm))[:,::decim]
    del brainmask

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Load and slice through EPI image.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ## Load data.
    obj = nib.load(os.path.join(mri_dir, subject, 'arc_001', '001', 'fmcpr.nii'))
    _,_,_,n_acq = obj.shape
    
    ## Preallocoate space for timeseries.
    gmts = np.zeros((n_acq, gm.shape[-1]))
    wmts = np.zeros((n_acq, wm.shape[-1]))
    
    for n in range(n_acq):

        ## Slice image.
        acq = obj.dataobj[..., n]
        
        ## Store grey matter.
        gmts[n] += acq[gm[0],gm[1],gm[2]]
        
        ## Store white matter.
        wmts[n] += acq[wm[0],wm[1],wm[2]]
        
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Preprocessing data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    ## Construct highpass filter.
    sfreq = 1. / tr
    high_pass = 1. / high_pass
    iir_params = dict(order=2, ftype='butter', output='sos') # Following Power et al. (2014)
    iir_params = construct_iir_filter(iir_params, high_pass, None, sfreq, 'highpass', return_copy=False)  
    
    ## Filter data.
    gmts = filter_data(gmts.T, sfreq, high_pass, None, method='iir', iir_params=iir_params, verbose=False)
    wmts = filter_data(wmts.T, sfreq, high_pass, None, method='iir', iir_params=iir_params, verbose=False)
    
    ## De-mean (we'll save this for later.)
    #gmts = np.apply_along_axis(demean, 1, gmts)
    #wmts = np.apply_along_axis(demean, 1, wmts)
    
    ## Re-organize (center outwards).
    gmts = gmts[ np.argsort( np.power( np.apply_along_axis(demean, 1, gm), 2 ).sum(axis=0) ) ]
    wmts = gmts[ np.argsort( np.power( np.apply_along_axis(demean, 1, wm), 2 ).sum(axis=0) ) ]
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Save data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    f = os.path.join(out_dir, '%s_arc_qc_data' %subject)
    np.savez_compressed(f, gm=gmts, wm=wmts, iir_params=iir_params )
    del gmts, wmts, obj
    
print 'Done.'

### Compute Summary Statistics of and Visualize Motion

In [ ]:
import os
import numpy as np
import pylab as plt
from collections import defaultdict
from pandas import DataFrame, read_csv
def demean(arr): return arr - arr.mean()

mri_dir = '/autofs/space/lilli_002/users/DARPA-ARC'
behavior_dir = '/space/sophia/2/users/DARPA-Behavior/arc/csv'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

thresholds = [0.5,0.9,1.3]
selected_threshold = 0.9
tr = 1.75

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

info = read_csv('demographics.csv')
subjects = info.loc[~info.Exlude,'Subject'].as_matrix()
stats = defaultdict(list)

for subject in subjects:
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Load and prepare MRI data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ## Load gray/white matter timeseries.
    npz = np.load('fmri_motion/%s_arc_qc_data.npz' %subject)
    gm = np.apply_along_axis(demean, 1, npz['gm'])
    wm = np.apply_along_axis(demean, 1, npz['wm'])
    
    ''' A NOTE ON MOTION DATA.
    1) Understanding Freesurfer motion outputs: https://mail.nmr.mgh.harvard.edu/pipermail//freesurfer/2013-May/030273.html
    2) Understanding angular displacement: https://en.wikipedia.org/wiki/Angular_displacement
    3) Understanding framewise displacement: See Power 2012, 2014
    '''

    ## Read motion data.
    mc = os.path.join(mri_dir, subject, 'arc_001', '001', 'fmcpr.mcdat')
    mc = np.loadtxt(mc)[:,1:7]

    ## Invert angular displacement.
    mc[:,:3] = np.deg2rad(mc[:,:3]) # Convert degrees to radians
    mc[:,:3] *= 50                  # Convert radians to mm [Following Power 2012, we assume a head ~ sphere w/ r=50mm]

    ## Compute framewise displacement (See Power 2012, 2014).
    fd = np.insert( np.abs( np.diff(mc, axis=0) ).sum(axis=1), 0, 0 )

    ## Compute absolute displacement. 
    rot = ( np.abs( mc - mc[0] )[:,:3] ).sum(axis=1)
    trans = ( np.abs( mc - mc[0] )[:,3:] ).sum(axis=1)
    
    ## Compute volumes to remove.
    rejections = np.zeros_like(thresholds)
    for n, threshold in enumerate(thresholds): rejections[n] += (fd >= threshold).sum()
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Load and prepare behavior data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    ## Load behavior CSV.
    df = read_csv(os.path.join(behavior_dir, '%s_arc_mri_ser-1' %subject))
    
    ## Create onsets of each TR.
    onsets = np.cumsum( np.ones_like(fd) * tr )
    onsets = np.insert(onsets, 0, 0)

    ## Define onsets/offsets of trials & shocks.
    trial_starts = df.RiskOnset.as_matrix()
    trial_ends = np.append(df.FixOnset[1:], trial_starts[-1] + 5.25)
    shock_starts = df.ShockOnset[~df.ShockOnset.isnull()].as_matrix()
    shock_ends = np.array([trial_ends[np.argmin((trial_ends - s)**2)] for s in shock_starts])

    ## Digitize onsets/offsets.
    trial_starts = np.digitize(trial_starts, onsets)
    trial_ends = np.digitize(trial_ends, onsets)
    shock_starts = np.digitize(shock_starts, onsets)
    shock_ends = np.digitize(shock_ends, onsets)
    
    ## Make boxcars for plotting.
    trials = np.zeros_like(fd)
    for i,j in zip(trial_starts,trial_ends): trials[i:j+1] += 1 
        
    shocks = np.zeros_like(fd)
    for i,j in zip(shock_starts,shock_ends): shocks[i:j+1] += 1 

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Calculate summary statistics.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    stats['Subject'] += [subject]
    
    ## Add motion information.
    stats['Abs_Disp_Rot']   += [rot.max()]
    stats['Abs_Disp_Trans'] += [trans.max()]
    stats['FD_mean'] += [fd.mean()]
    stats['FD_sd']   += [fd.std()]
    stats['FD_max']  += [fd.max()]
    
    ## Calculate number of rejections.
    fd_index, = np.where(fd >= selected_threshold)
    n_reject = len(fd_index)
    stats['FD_reject'] += [n_reject]
    
    ## Calculate fraction of rejected displacements across all 
    ## instances of a portion of the run.
    stats['FD_frac_task']  += [((trials) * (fd >= selected_threshold)).mean()]
    stats['FD_frac_rest']  += [((1 - trials) * (fd >= selected_threshold)).mean()]
    stats['FD_frac_shock'] += [((shocks) * (fd >= selected_threshold)).mean()]
    
    ## Calculate percentages of rejection displacesments
    ## within given categories (non-shock task, shock, rest)
    stats['FD_perc_rest']    += [(1-trials)[fd_index].sum() / float(n_reject) ]
    stats['FD_perc_shock']   += [shocks[fd_index].sum() / float(n_reject) ]
    stats['FD_perc_task_ns'] += [1 - stats['FD_perc_rest'][-1] - stats['FD_perc_shock'][-1]]
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Plotting.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ## Initialize figure.
    fig = plt.figure(figsize=(18,9), dpi=120)
    nrow = 11
    
    ## Plot absolute motion.
    ax = plt.subplot2grid((nrow,1),(0,0)) 
    ax.plot(rot, linewidth=2, label='Rot', color='#4daf4a')
    ax.plot(trans, linewidth=2, label='Trans', color='#984ea3')
    ax.legend(loc=2, frameon=False, borderpad=0, labelspacing=0.1, handletextpad=0.1)
    ax.set_xlim(0,fd.shape[0])
    ax.set_xticks([])
    ax.set_ylabel('Dist (mm)', fontsize=10)
    ax.set_ylim(0,10)
    ax.set_yticks([0,5,10])
    ax.tick_params(axis='y', which='major', labelsize=6)
    ax.set_title('%s ARC Motion' %subject.upper(), fontsize=24)
    
    ## Plot shocks.
    ax = plt.subplot2grid((nrow,1),(1,0))
    ax.plot(shocks*0.5, linewidth=2, color='k')
    ax.set_xticks([])
    ax.set_xlim(0,fd.shape[0])
    ax.set_ylim(0,1)
    ax.set_yticks([])
    ax.set_ylabel('Shocks\n', fontsize=10)
    
    ## Plot framewise displacement.
    ax = plt.subplot2grid((nrow,1),(2,0),rowspan=2)
    ax.plot(fd, linewidth=2, color='k')
    for t, r, c in zip(thresholds, rejections, ['#e41a1c', '#377eb8','#4daf4a']): 
        ax.plot(np.ones_like(fd)*t, linewidth=2, linestyle='--', alpha=0.9, label='%s (%s volumes)' %(t,int(r)), color=c)
    ax.legend(loc=2, frameon=False, borderpad=0, labelspacing=0.1, handletextpad=0.1)
    ax.set_xlim(0,fd.shape[0])
    ax.set_xticks([])
    ax.set_ylabel('Dist (mm)', fontsize=11)
    ax.set_ylim(0,2.5)
    ax.tick_params(axis='y', which='major', labelsize=8)
    
    ## Plot gray matter.
    ax = plt.subplot2grid((nrow,1),(4,0),rowspan=4)
    cbar = ax.imshow(gm, aspect='auto', interpolation='none', origin='lower', cmap='bone', vmin=-50, vmax=50)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_ylabel('Gray Matter', fontsize=14)

    ## Plot white matter.
    ax = plt.subplot2grid((nrow,1),(8,0),rowspan=3)
    cbar = ax.imshow(wm, aspect='auto', interpolation='none', origin='lower', cmap='bone', vmin=-50, vmax=50)
    ax.set_yticks([])
    ax.set_ylabel('White Matter', fontsize=14)
    ax.set_xlabel('Acquisitions', fontsize=16)

    plt.subplots_adjust(left=0.05, right=0.975, top=0.95, bottom=0.06, hspace=0.1)
    plt.savefig('plots/motion/%s_motion_raw.png' %subject)
    #plt.show()
    plt.close('all')
    

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Save summary statistics.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
stats = DataFrame(stats)
stats = stats[['Subject', 'Abs_Disp_Rot', 'Abs_Disp_Trans', 'FD_mean', 'FD_sd', 'FD_max',
               'FD_reject', 'FD_frac_task', 'FD_frac_shock', 'FD_frac_rest',
               'FD_perc_task_ns',  'FD_perc_shock', 'FD_perc_rest']]
stats.to_csv('fmri_motion/motion_stats.csv', index=False)

print 'Done.'

In [ ]:
rejections

### Present Motion Table

In [ ]:
from pandas import read_csv
df = read_csv('motion/motion_stats.csv')
df.describe()

# Section 3: fMRI Analysis (First Levels)

## Generate Subject Task Regressors

In [ ]:
import numpy as np
import pylab as plt
from pandas import read_csv
from scipy.special import gammaln
from statsmodels.stats.outliers_influence import variance_inflation_factor
mri_dir = 'fmri_first_levels'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Results file.
results_file = 'arc_hierarchical_add_FINAL2_regressors'

## Define contrasts.
conditions = ['Delib','DelibMod','Antcp','AntcpMod','Shock']
n_conditions = len(conditions)

## Timing information.
n_acq = 977
tr = 1.75
sfreq = 1e2

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define useful functions.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

def spm_hrf(RT, P=None, fMRI_T=16):
    p = np.array([6, 16, 1, 1, 6, 0, 32], dtype=float)
    if P is not None:
        p[0:len(P)] = P

    _spm_Gpdf = lambda x, h, l: np.exp(h * np.log(l) + (h - 1) * np.log(x) - (l * x) - gammaln(h))
    # modelled hemodynamic response function - {mixture of Gammas}
    dt = RT / float(fMRI_T)
    u = np.arange(0, int(p[6] / dt + 1)) - p[5] / dt
    with np.errstate(divide='ignore'):  # Known division-by-zero
        hrf = _spm_Gpdf(u, p[0] / p[2], dt / p[2]) - _spm_Gpdf(u, p[1] / p[3],
                                                               dt / p[3]) / p[4]
    idx = np.arange(0, int((p[6] / RT) + 1)) * fMRI_T
    hrf = hrf[idx]
    hrf = hrf / np.sum(hrf)
    return hrf

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~# 

results_file = 'stan_results/%s.csv' %results_file
df = read_csv(results_file)

for subject in np.unique(df.Subject):
    
    print subject,

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Initialize regressors.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~# 
    
    ## Setup timing information.
    total_time = n_acq * tr
    times = np.arange(0, total_time+1./sfreq, 1./sfreq)
    n_times = times.shape[0]

    ## Initialize boxcars.
    neural_signal = np.zeros((n_conditions,n_times))
    
    ## Extract information.
    extract_cols = ['ddb','RiskType','ResponseType','RiskOnset','RT','ShockOnset']
    extract_cols = df.loc[df.Subject==subject, extract_cols].copy().as_matrix()
    DDB, Risk, Choice, TrialOnset, RT, ShockOnset = extract_cols.T.round(int(np.log10(sfreq)))
    
    ## Prepare information.
    RT += 0.5                     # Reaction time does not factor 0.5s of risk presentation.
    RT = np.where(np.isnan(RT), 3.5, RT)
    DDB = np.where(np.isnan(DDB),0,DDB)
    Risk = np.where(Risk<2,0.1,np.where(Risk<3,0.5,0.9))
    Choice = np.where(np.isnan(Choice),0,Choice)
    ShockOnset = ShockOnset[~np.isnan(ShockOnset)]

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Generate decision-making boxcars.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~# 

    for onset, duration, parmod in zip(TrialOnset, RT, DDB): 
        mask = (times >= onset) & (times <= onset + duration)
        neural_signal[0,mask] += 1         # Deliberation (intercept)
        neural_signal[1,mask] += parmod    # Deliberation (DDB)

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Generate expectation boxcars.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~# 
    
    ## Add anticipation information.
    antcp_onset  = TrialOnset + RT
    antcp_offset = TrialOnset + 3.5 + 1.25
    
    for onset, offset, choice, risk in zip(antcp_onset, antcp_offset, Choice, Risk):
        mask = (times >= onset) & (times <= offset)
        neural_signal[2,mask] += 1
        neural_signal[3,mask] += choice * risk
    
    ## Add shock information.
    for onset in ShockOnset:
        mask = (times >= onset) & (times <= onset + 0.5)
        neural_signal[-1,mask] += 1

    ## Perform convolution.
    hrf = spm_hrf(1./sfreq)
    bold_signal = np.apply_along_axis(np.convolve, 1, neural_signal, v=hrf)
    bold_signal = bold_signal[:,:neural_signal.shape[-1]] # Set back to original length.
    
    ## Downsample to start of TR.
    tr_onsets = np.insert( np.cumsum( np.ones(n_acq-1)*tr ), 0, 0 )
    ds = np.in1d(times, tr_onsets)
    if not ds.sum() == n_acq: raise ValueError('Oh noes!')
    bold_signal = bold_signal[:,ds]
    
    ## Normalize regressors (max height=1).
    bold_signal = (bold_signal.T / bold_signal.max(axis=1)).T

    ## Save task regressors.
    for arr, label in zip(bold_signal, conditions):

        f = '%s/%s/arc_001/001/FINAL2.%s.par' %(mri_dir,subject,label)
        try: np.savetxt(f, arr[:,np.newaxis], fmt='%s')
        except IOError: pass
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Compute and plot VIF.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~# 
    
    ## Compute VIF.
    bold_signal = bold_signal.T
    vif = np.array([variance_inflation_factor(bold_signal,i) for i in range(n_conditions)])
    if np.any(np.isinf(vif)): raise ValueError('Oh noes! Check VIF!')
    
    ## Open figure.
    fig = plt.figure(figsize=(18,6))
    colors = ['#377eb8','#4daf4a','#e41a1c','#984ea3','#ff7f00','#386cb0','#e7298a','#66a61e']

    ## Plot VIF.
    ax = plt.subplot2grid((2,3),(0,0),rowspan=2)
    ax.bar(np.arange(n_conditions), vif, 0.9, color='#7ec0ee')
    ax.set_xlim(-0.1)
    ax.set_xticks(np.arange(n_conditions)+0.45)
    ax.set_xticklabels(conditions)
    ax.set_ylim(0,10)
    ax.set_ylabel('VIF', fontsize=20)
    ax.set_title('%s Collinearity' %subject.upper(), fontsize=24)

    ## Plot decision-making regressors.
    ax = plt.subplot2grid((2,3),(0,1),colspan=2)
    for arr, label, color in zip(bold_signal.T[:2], conditions[:2], colors[:2]):
        ax.plot(tr_onsets, arr, linewidth=2, color=color, alpha=0.8, label=label)
    ax.legend(loc=2, bbox_to_anchor=(1.0,0.7), frameon=False, borderpad=0, handletextpad=0.1)
    ax.set_xticks([])
    ax.set_xlim(0,180)
    ax.set_yticks([])
    ax.set_title('Decision Making', fontsize=24)

    ## Plot anticipation regressors.
    ax = plt.subplot2grid((2,3),(1,1),colspan=2)
    for arr, label, color in zip(bold_signal.T[2:], conditions[2:], colors[2:]):
        ax.plot(tr_onsets, arr, linewidth=2, color=color, alpha=0.8, label=label)
    ax.legend(loc=2, bbox_to_anchor=(1.0,0.8), frameon=False, borderpad=0, handletextpad=0.1)
    ax.set_xlim(0,180)
    ax.set_xlabel('Time (s)', fontsize=16)
    ax.set_yticks([])
    ax.set_title('Anticipation', fontsize=24)

    plt.subplots_adjust(left=0.05, wspace=0.05, hspace=0.3)
    plt.savefig('plots/vif/%sreg2_%s.png' %(n_conditions,subject))
    plt.close('all')

print 'Done.'

## Generate Subject Timepoint Censors

In [ ]:
import os
import numpy as np
from pandas import read_csv
from scipy.signal import detrend
from sklearn.decomposition import PCA
from statsmodels.stats.outliers_influence import variance_inflation_factor
mri_dir = 'fmri_first_levels'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Timing information.
n_acq = 977
tr = 1.75

## Scrubbing parameters.
thresholds = [0.0, 0.5, 0.7, 0.9, 1.1, 1.3]

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Define TR onsets.
tr_onsets = np.insert( np.cumsum( np.ones(n_acq - 1) * tr ), 0, 0 )

## Get subjects list.
info = read_csv('demographics.csv')
subjects = info.loc[~info.Exlude, 'Subject'].as_matrix()
info = open('fmri_second_levels/nuisance_info.csv','w')
info.write('Subject,n_mc,FD=0.0,FD=0.5,FD=0.7,FD=0.9,FD=1.1,FD=1.3\n')

for subject in subjects:
    
    info.write('%s,' %subject)
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Compute framewise displacement.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    ## Read motion data.
    mc = os.path.join(mri_dir, subject, 'arc_001', '001', 'fmcpr.mcdat')
    mc = np.loadtxt(mc)[:,1:7]

    ## Invert angular displacement.
    fd = mc.copy()
    fd[:,:3] = np.deg2rad(fd[:,:3]) 
    fd[:,:3] *= 50

    ## Compute framewise displacement (See Power 2012, 2014).
    fd = np.insert( np.abs( np.diff(fd, axis=0) ).sum(axis=1), 0, 0 )

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Compute motion regressors.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    ## Remove trends.
    mc = detrend(mc, axis=0, type='constant')
    mc = detrend(mc, axis=0, type='linear')
    
    ## Perform PCA.
    pca = PCA(n_components=6)
    mc = pca.fit_transform(mc)
    
    ## Take only the number of components explaining 90% of the variance.
    varexp = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(varexp >= 0.9) + 1
    mc = mc[:,:n_components]
    
    ## Save motion regressor.
    f = '%s/%s/arc_001/001/FINAL2.mc.par' %(mri_dir,subject)
    np.savetxt(f, mc, fmt='%s')
    info.write('%s,' %n_components)
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Write scrubbers.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    for threshold in thresholds:
        
        ## Find threshold violations.
        if not threshold: ix, = np.where(fd >= np.inf)
        else: ix, = np.where(fd >= threshold)
                
        ## Save.
        info.write('%s,' %len(ix))
        f = '%s/%s/arc_001/001/FINAL2.censor.%s.par' %(mri_dir,subject,threshold)
        if len(ix): np.savetxt(f, tr_onsets[ix,np.newaxis], fmt='%s')

    info.write('\n')

info.close()
print 'Done.'

## Censor Analysis: Precompute F maps

In [ ]:
import os
import numpy as np
import nibabel as nib
from pandas import read_csv
mri_dir = 'fmri_first_levels/concat-sess/FINAL2'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

sm = 6
thresholds = [0.0,0.5,0.7,0.9,1.1,1.3]
spaces = ['lh','rh','mni305']

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Setup for WLS.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Load subject information.
info = read_csv('demographics.csv')
info = info[~info.Exlude].reset_index()
n_subj, _ = info.shape

## Build Design Matrix.
X = np.zeros((n_subj,2))
X[:,0] = 1                                        # Intercept
X[:,1] = np.where(info.Scanner == 'Trio', 0, 1)   # Scanner
n_subj, n_pred = X.shape

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

def wls(X,Y,W):
    B = np.linalg.inv(X.T.dot(W).dot(X)).dot(X.T).dot(W).dot(Y)
    ssr = W.dot( np.power(Y - np.dot(X,B),2) ).sum()
    scale = ssr / (n_subj - n_pred)
    cov_p = np.linalg.inv(X.T.dot(W).dot(X)) * scale
    F = np.power(B[0],2) * np.power(cov_p[0,0],-1)
    return B[0], F

for space in spaces:
    
    for fd in thresholds:

        print space, fd
        
        results_dir = os.path.join(mri_dir, 'FINAL2.%s.%s.%s' %(sm,fd,space))
        
        ## Load data.
        ces = nib.load(os.path.join(results_dir, 'FINAL2.Delib.par', 'ces.nii.gz')).get_data().squeeze()
        cesvar = nib.load(os.path.join(results_dir, 'FINAL2.Delib.par', 'cesvar.nii.gz')).get_data().squeeze()
        affine = nib.load(os.path.join(results_dir, 'FINAL2.Delib.par', 'ces.nii.gz')).affine
        
        ## Reshaping of MNI305 data.
        if space == 'mni305':
            x,y,z,n_subj = ces.shape
            ces = ces.reshape(x*y*z,n_subj)
            cesvar = cesvar.reshape(x*y*z,n_subj)
        
        ## Preallocate arrays for results.
        cesvar = np.abs(1./cesvar)
        include, = np.where(~np.isinf(cesvar).sum(axis=1).astype(bool))
        Fmap = np.repeat(np.nan, ces.shape[0])
        
        ## Perform WLS.
        for i in include:
            
            ## Update variables
            Y = ces[i]
            W = np.diag(cesvar[i])
            _, Fmap[i] = wls(X,Y,W)
        
        ## Reshape.
        if space == 'mni305': Fmap = Fmap.reshape(x,y,z)
        
        ## Save.
        for _ in range(4 - len(Fmap.shape)): Fmap = np.expand_dims(Fmap, -1)
        obj = nib.Nifti1Image(Fmap, affine)
        nib.save(obj, os.path.join(results_dir, 'FINAL2.Delib.par', 'F.nii.gz'))
        
print 'Done.'

## Censor Analysis: Determine Optimal Threshold
Based on the methods from [Siegal et al. (2014)](https://www.ncbi.nlm.nih.gov/pubmed/23861343): *Statistical Improvements in Functional Magnetic Resonance Imaging Analyses Produced by Censoring High-Motion Data Points*.

In [ ]:
import os, shutil
import numpy as np
import nibabel as nib
import pylab as plt
import seaborn as sns
from pandas import DataFrame
from mne import read_surface, grow_labels, spatial_tris_connectivity, set_log_level
from mne.stats.cluster_level import _find_clusters as find_clusters
from scipy.stats import f_oneway
from scipy.stats import f as fdist
from scipy.ndimage import measurements
set_log_level(verbose=False)
sns.set_style('white')
sns.set_context('notebook', font_scale=1.5)
%matplotlib inline

fs_dir = 'recons'
mri_dir = 'fmri_first_levels/concat-sess/FINAL2'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## I/O paramters.
sm = 6
contrast = 'Delib'
censor = True    # {True = Include blobs from all overlays, 
                 # False = Include blobs from only no-center}

## Cluster parameters.
cluster_dict = dict(lh = [0.01, 100], rh = [0.01, 100],
                    mni305 = [0.01, 20])

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Define iterators.
spaces = ['lh','rh','mni305']
thresholds = [0.0, 0.5, 0.7, 0.9, 1.1, 1.3]

info = []
for n, space in enumerate(spaces):

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Load data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    overlays = []
    for fd in thresholds: 
        obj = nib.load(os.path.join(mri_dir, 'FINAL2.%s.%s.%s' %(sm,fd,space), 'FINAL2.%s.par' %contrast, 'F.nii.gz'))
        overlays.append( obj.get_data().squeeze() )
    overlays = np.array(overlays)

    ## Make average map.
    if censor: average = overlays.mean(axis=0)
    else: average = overlays[0].copy()

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Identify clusters.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    p_value, min_cluster = cluster_dict[space]
    min_value = fdist.isf(p_value, 1, 26)
    
    if space == 'mni305':

        masked_average = average > min_value
        clusters, n_clusters = measurements.label( masked_average )
        clusters = [np.where(clusters==n) for n in np.arange(n_clusters)+1 if (clusters==n).sum() > min_cluster]
        
    else:

        ## Prepare surface information.
        _, tris = read_surface(os.path.join(fs_dir, 'fsaverage', 'surf', '%s.white' %space))
        connectivity = spatial_tris_connectivity(tris)
        include = np.invert(np.isnan(average).astype(bool))
        
        ## Identify clusters (clusters already sorted by size).
        clusters, _ = find_clusters(average, min_value, tail=1, connectivity=connectivity, include=include)
        clusters = [c for c in clusters if len(c) > min_cluster]
        
    print '%s clusters identified for %s.' %(len(clusters), space)

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Average across labels / spheres.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    for i, fd in enumerate(thresholds):
                    
        for j, c in enumerate(clusters):
            
            fscore = np.nanmean(overlays[i][c])
            info.append([fd,space,j,fscore])

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Compute statistics.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
info = DataFrame(np.array(info), columns=('FD','Space','Label','Fscore'))
info['Fscore'] = info.Fscore.astype(float)
print f_oneway(*[info.loc[info.FD==fd,'Fscore'].as_matrix() for fd in np.unique(info.FD)])
print info.groupby(['FD',]).Fscore.mean()

## Plot.
g = sns.factorplot('Space', 'Fscore', 'FD', info, kind='bar', ci=None, size=4, aspect=2);
g.ax.set_ylim(12,16);

# Section 4: fMRI Analysis (Second-Levels)


## Precompute Permutations
Based on intial calculations, we assume one full loop of WLS + TFCE will take ~17s. We will submit jobs of 100 iterations (approx. 30 minutes time on cluster).

In [ ]:
import os
import numpy as np
np.random.seed(47404)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

n_subj = 28
n_permutations = 5000
inc = 100

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Generate permutations.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

permutations = []
while True:
    arr = np.random.choice([1,-1],n_subj,replace=True)
    if not np.any(np.apply_along_axis(np.array_equal, 0, permutations, arr)): 
        permutations.append(arr)
    if len(permutations) >= n_permutations: 
        break 
        
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Save.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

permutations = np.array(permutations)
index = np.arange(0,n_permutations+1,inc)
for n, ix in enumerate(index[1:]): 
    np.save(os.path.join('fmri_second_levels', 'permutations', 'sign_flips_%s' %(n+1)), permutations[ix-inc:ix])
            
print 'Done.'

## Make Surface Masks

In [ ]:
import os
import numpy as np
import nibabel as nib
from mne import read_label, read_surface, spatial_tris_connectivity, set_log_level
set_log_level(verbose=False) 
subj_dir = '/space/lilli/1/users/DARPA-Recons/fscopy'
mri_dir = '/autofs/space/lilli_002/users/DARPA-ARC/NN_bayes_2016/FINAL6/'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

label_dir = os.path.join(subj_dir, 'label', 'dkt40')

roi_list = ['caudalanteriorcingulate', 'rostralanteriorcingulate', 'posteriorcingulate',
            'superiorfrontal', 'medialorbitofrontal', 'rostralmiddlefrontal', 'caudalmiddlefrontal',
            'parsopercularis', 'parstriangularis', 'parsorbitalis', 'lateralorbitofrontal', 'insula']

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Make labels.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~# 

for hemi in ['lh', 'rh']:
    
    ## Assemble and merge labels.
    label = []
    for roi in roi_list: label.append(read_label(os.path.join(label_dir,'%s-%s.label' %(roi,hemi))))
    label = np.sum(label)
    
    ## Save label.
    label.name = 'arc-%s' %hemi
    label.save('fmri_second_levels/arc-%s.label' %hemi)
    
    ## Load surface.
    _, tris = read_surface(os.path.join(subj_dir, 'surf', '%s.white' %hemi))
    mapping = np.in1d(np.unique(tris),label.vertices)
    
    ## Reduce triangles to those in label.
    ix = np.all(np.apply_along_axis(np.in1d, 0, tris, label.vertices), axis=1)
    tris = tris[ix]

    ## Compute connectivity.
    coo = spatial_tris_connectivity(tris, remap_vertices=True)
    np.savez('fmri_second_levels/%s_connectivity' %hemi, data = coo.data, row = coo.row,
             col = coo.col, shape = coo.shape, mapping=mapping, vertices=label.vertices)
    
print 'Done.'

## Make Volume Mask

In [ ]:
import os
import numpy as np
import nibabel as nib
from scipy.sparse import coo_matrix
lut = '/usr/local/freesurfer/stable5_3_0/FreeSurferColorLUT.txt'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Define subcortical structures of interest.
roi_dict = {18:'Left-Amygdala', 11:'Left-Caudate', 17:'Left-Hippocampus', 12:'Left-Putamen', 
            54:'Right-Amygdala', 50:'Right-Caudate', 53:'Right-Hippocampus', 51:'Right-Putamen'}

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Create mask.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Load aseg.
aseg = '/space/lilli/1/users/DARPA-Recons/fsaverage/mri.2mm/aseg.mgz'
aseg = nib.load(aseg).get_data()

## Find all voxels in ROI list. Get corresponding labels.
mapping = np.in1d(aseg, roi_dict.keys()).reshape(aseg.shape)
voxels = np.where(mapping)
names = np.array([roi_dict[i] for i in aseg[voxels]])
voxels = np.vstack(voxels).T

## Initialize connectivity matrix.
n_voxels, _ = voxels.shape
coo = np.zeros([n_voxels,n_voxels], dtype=int)

## Iteratively test for adjacency.
## Here we use 6-lattice connectivity (up,down,forward,backward,left,right).
for n in range(n_voxels):
    diff = np.linalg.norm(voxels - voxels[n], axis=1)
    M, = np.where(diff==1.)
    for m in M: coo[n,m] = 1 
coo = coo_matrix(coo)
    
## Save.
np.savez('fmri_second_levels/mni305_connectivity', data = coo.data, row = coo.row,
         col = coo.col, shape = coo.shape, mapping=mapping, voxels=voxels, names=names)
print 'Done.'

## Extract Mean Signal from ROIs
Necessary for computing percent signal change down the line.

In [ ]:
import os
import numpy as np
import nibabel as nib
from pandas import read_csv
mri_dir = 'fmri_first_levels/'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

sm = 6
fd = 0.9

n_acq = 977
tr = 1.75

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Get subjects list.
info = read_csv('demographics.csv')
subjects = info.loc[~info.Exlude, 'Subject'].as_matrix()

## Define TR onsets.
tr_onsets = np.insert( np.cumsum( np.ones(n_acq - 1) * tr ), 0, 0 )

mean_signal = dict()
for space in ['lh','rh','mni305']:
    
    print space,
    
    ## Load masks.
    npz = np.load('fmri_second_levels/%s_connectivity.npz' %space)
    include = npz['mapping']
    
    ## Preallocate space.
    ms = np.zeros([len(subjects), include.sum()])
    
    ## Iterate over subjects.
    for n, subject in enumerate(subjects):
        
        ## Load data.
        subj_dir = os.path.join(mri_dir, subject, 'arc_001', '001')
        if space == 'mni305': f = os.path.join(subj_dir,'fmcpr.sm%s.%s.2mm.b0dc.nii.gz' %(sm,space))
        else: f = os.path.join(subj_dir,'fmcpr.sm%s.fsaverage.%s.b0dc.nii.gz' %(sm,space))
        data = nib.load(f).get_data()
        
        ## Censor data. Average across acquisitions.
        try: censor = np.loadtxt(os.path.join(subj_dir, 'FINAL.censor.%s.par' %fd))
        except IOError: censor = []
        censor = np.invert(np.in1d(tr_onsets, censor))
        
        data = data[include,...].squeeze()
        data = data[...,censor].mean(axis=1)
        
        ## Append.
        ms[n] = data
    
    ## Store in dictionary.
    mean_signal[space] = ms
    
## Save.
f = 'fmri_second_levels/mean_signal'
np.savez_compressed(f, lh = mean_signal['lh'], rh = mean_signal['rh'], mni305 = mean_signal['mni305'])
print 'Done.'

## Assemble Data

In [ ]:
import os
import numpy as np
import nibabel as nib
from mne import read_label
from pandas import read_csv
mri_dir = 'fmri_first_levels/concat-sess/FINAL2/'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

sm = 6
spaces = ['lh','rh','mni305']
thresholds = [0.9]
contrasts = ['Delib', 'DelibMod', 'Antcp', 'AntcpMod', 'Shock']

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

for space in spaces:

    ## Load masks.
    npz = np.load('fmri_second_levels/%s_connectivity.npz' %space)
    include = npz['mapping']
    
    for fd in thresholds:
                
        results_dir = os.path.join(mri_dir, 'FINAL2.%s.%s.%s' %(sm,fd,space))
            
        for contrast in contrasts:
        
            ## Make save directory.
            out_dir = os.path.join('fmri_second_levels', 'FINAL2.%s.%s.%s.%s' %(sm,fd,space,contrast))
            if not os.path.isdir(out_dir): os.makedirs(out_dir)
    
            ## Load data.
            ces = nib.load(os.path.join(results_dir, 'FINAL2.%s.par' %contrast, 'ces.nii.gz')).get_data()
            cesvar = nib.load(os.path.join(results_dir, 'FINAL2.%s.par' %contrast, 'cesvar.nii.gz')).get_data()
            affine = nib.load(os.path.join(results_dir, 'FINAL2.%s.par' %contrast, 'ces.nii.gz')).affine
            
            ## Masking.
            ces = ces[include,...]
            cesvar = cesvar[include,...]
                    
            ## Save.
            np.savez_compressed(os.path.join(out_dir, 'first_levels'), ces=ces.squeeze(), cesvar=cesvar.squeeze())
            np.save(os.path.join(out_dir, 'affine'), affine)

print 'Done.'

## Perform WLS Permutations

In [ ]:
import os, sys
import numpy as np
from pandas import read_csv
from scipy.sparse import coo_matrix
from mne.stats.cluster_level import _find_clusters as find_clusters
root_dir = '/space/sophia/2/users/DARPA-Behavior/notebooks/bayes/decision_making/NN_bayes_2016'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## I/O parameters.
sm = 6
fd = 0.9
space = 'mni305'
contrast = 'Delib'

## Permutation parameters.
permutations = 0

## TFCE parameters.
threshold = dict(start=0.1, step=0.1, h_power=2, e_power=0.5)
tail = 0
max_step = 1

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Load and prepare data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

def load_sparse_coo(filename):
    npz = np.load(filename)
    M,N = npz['shape']
    return coo_matrix( (npz['data'], (npz['row'],npz['col'])), (M,N) )

out_dir = os.path.join(root_dir, 'fmri_second_levels', 'FINAL.%s.%s.%s.%s' %(sm,fd,space,contrast))

## Load data.
npz = np.load(os.path.join(out_dir, 'first_levels.npz'))
ces = npz['ces']
cesvar = np.abs( 1. / npz['cesvar'] )

## Define indices.
connectivity = load_sparse_coo(os.path.join(root_dir, 'fmri_second_levels', '%s_connectivity.npz' %space))
index,  = np.where(~np.isinf(cesvar).sum(axis=1).astype(bool))
include = ~np.isinf(cesvar).sum(axis=1).astype(bool)
    
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Setup for permutation testing.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Load subject information.
info = read_csv(os.path.join(root_dir, 'demographics.csv'))
info = info[~info.Exlude].reset_index()
n_subj, _ = info.shape

## Build Design Matrix.
X = np.zeros((n_subj,2))
X[:,0] = 1                                        # Intercept
X[:,1] = np.where(info.Scanner == 'Trio', 0, 1)   # Scanner
n_subj, n_pred = X.shape

## If specified, load precomputed sign flips.
if permutations: sign_flips = np.load(os.path.join(root_dir, 'fmri_second_levels', 'permutations', 'sign_flips_%s.npy' %permutations))
else: sign_flips = np.ones((1,n_subj))
n_shuffles = sign_flips.shape[0]

## Preallocate arrays for results.
shape = [n_shuffles] + list(ces.shape[:-1])
Bmap = np.zeros(shape)
Fmap = np.zeros(shape)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

'''
Following the instructions of Winkler et al. (2014), we use the Freedman and Lane (1983)
permutation procedure. This allows us to precompute a number of values ahead of time.

To understand the WLS computations, please see:
https://github.com/statsmodels/statsmodels/blob/master/statsmodels/base/model.py
https://github.com/statsmodels/statsmodels/blob/master/statsmodels/regression/linear_model.py
'''

def wls(X,Y,W):
    B = np.linalg.inv(X.T.dot(W).dot(X)).dot(X.T).dot(W).dot(Y)
    ssr = W.dot( np.power(Y - np.dot(X,B),2) ).sum()
    scale = ssr / (n_subj - n_pred)
    cov_p = np.linalg.inv(X.T.dot(W).dot(X)) * scale
    F = np.power(B[0],2) * np.power(cov_p[0,0],-1)
    return B[0], F

## Loop it!
for n, sf in enumerate(sign_flips):
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Compute statistics.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    for m in index:

        ## Update variables.
        W = np.diag(cesvar[m])
        Y = ces[m]
        
        ## Permute values.
        ## See Winkler et al. (2014), pg. 385
        ## To compute Hat Matrix, see: https://en.wikipedia.org/wiki/Projection_matrix and 
        Z = X[:,1:]
        ZZ = Z.dot( np.linalg.inv( Z.T.dot(W).dot(Z) ) ).dot(Z.T).dot(W)
        Rz = np.identity(n_subj) - ZZ
        Y = np.diag(sf).dot(Rz).dot(Y)
        
        ## Perform WLS.
        Bmap[n,m], Fmap[n,m] = wls(X,Y,W) 

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Perform TFCE.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    _, Fmap[n] = find_clusters(Fmap[n], threshold, tail=tail, connectivity=connectivity, 
                               include=include, max_step=max_step, show_info=False)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Save results.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#  

if not permutations: f = os.path.join(out_dir, '%s_%s_obs' %(space, contrast))
else: f = os.path.join(out_dir, '%s_%s_perm-%s' %(space, contrast, permutations)) 
np.savez_compressed(f, Bmap=Bmap, Fmap=Fmap)
    
print 'Done.'

## Perform FWE Corrections

In [ ]:
import os
import numpy as np
import nibabel as nib

def prepare_image(arr, space):
    npz = np.load('fmri_second_levels/%s_connectivity.npz' %space)
    image = np.zeros_like(npz['mapping'], dtype=float)
    
    if not space == 'mni305': 
        image[npz['vertices']] += arr
    else:
        x,y,z = npz['voxels'].T
        image[x,y,z] += arr
    
    for _ in range(4 - len(image.shape)): image = np.expand_dims(image,-1)
    return image

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

sm = 6
fd = 0.9
spaces = ['lh','rh','mni305']
contrasts = ['Delib','DelibMod','Antcp','AntcpMod','Shock']

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

permutations = np.arange(50) + 1

for contrast in contrasts:

    print contrast,
    
    for n, space in enumerate(spaces):
    
        out_dir = 'fmri_second_levels/FINAL2.%s.%s.%s.%s' %(sm, fd, space, contrast)
        
        ## Load true effects.
        npz = np.load(os.path.join(out_dir, '%s_%s_obs.npz' %(space,contrast)))
        Bmap = npz['Bmap'].squeeze()
        Fmap = npz['Fmap'].squeeze()
        
        ## Load permutations.
        Pmap = []
        for p in permutations: 
            npz = np.load(os.path.join(out_dir, '%s_%s_perm-%s.npz' %(space,contrast,p)))
            Pmap.append(npz['Fmap'])
        Pmap = np.concatenate(Pmap, axis=0)
        n_permutations, _ = Pmap.shape

        ## Compute p-values via FWE.
        p_values = np.ones_like(Fmap)
        for mp in Pmap.max(axis=1): p_values += mp > Fmap
        p_values /= n_permutations + 1.
        p_values = -np.log10(p_values) * np.sign(Bmap)
      
        ## Save maps.
        np.save(os.path.join(out_dir,'%s_%s_fwe' %(space,contrast)), p_values)
        for arr, name in zip([Bmap,Fmap,p_values],['beta','F','fwe']):
            image = prepare_image(arr, space)
            image = nib.Nifti1Image(image, np.load(os.path.join(out_dir,'affine.npy')))
            nib.save(image, os.path.join(out_dir, '%s.nii.gz' %name))
            
print 'Done.'

## Perform FDR Corrections
Not used, but programmed for example's sake.

In [ ]:
import os
import numpy as np
import nibabel as nib
from mne.stats import fdr_correction

def prepare_image(arr, space):
    npz = np.load('fmri_second_levels/%s_connectivity.npz' %space)
    image = np.zeros_like(npz['mapping'], dtype=float)
    
    if not space == 'mni305': 
        image[npz['vertices']] += arr
    else:
        x,y,z = npz['voxels'].T
        image[x,y,z] += arr
    
    for _ in range(4 - len(image.shape)): image = np.expand_dims(image,-1)
    return image

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

sm = 6
fd = 0.9
spaces = ['lh','rh','mni305']
contrasts = ['Delib','DelibMod','Antcp','AntcpMod','Shock']
contrasts = ['DelibMod']

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

permutations = np.arange(50) + 1

for contrast in contrasts:

    print contrast,
    FDR, signs = [], []
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Compute p-values within spaces.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    for n, space in enumerate(spaces):
    
        out_dir = 'fmri_second_levels/FINAL.%s.%s.%s.%s' %(sm, fd, space, contrast)
        
        ## Load true effects.
        npz = np.load(os.path.join(out_dir, '%s_%s_obs.npz' %(space,contrast)))
        Bmap = npz['Bmap'].squeeze()
        Fmap = npz['Fmap'].squeeze()
        
        ## Load permutations.
        Pmap = []
        for p in permutations: 
            npz = np.load(os.path.join(out_dir, '%s_%s_perm-%s.npz' %(space,contrast,p)))
            Pmap.append(npz['Fmap'])
        Pmap = np.concatenate(Pmap, axis=0)
        n_permutations, _ = Pmap.shape

        ## Compute p-values via FWE.
        p_values = (Pmap >= Fmap).sum(axis=0) + 1.
        p_values /= n_permutations + 1.
        FDR.append(p_values)
        signs.append(np.sign(Bmap))
    
        ## Save maps.
        for arr, name in zip([Bmap,Fmap],['beta','F']):
            image = prepare_image(arr, space)
            image = nib.Nifti1Image(image, np.load(os.path.join(out_dir,'affine.npy')))
            nib.save(image, os.path.join(out_dir, '%s.nii.gz' %name))        

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Perform FDR corrections.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
            
    ## Assemble info.
    indices = np.concatenate([np.ones_like(arr) * n for n, arr in enumerate(FDR)])
    FDR = np.concatenate(FDR)
    signs = np.concatenate(signs)

    ## Perform FDR correction.
    FDR[np.where(signs)] = fdr_correction(FDR[np.where(signs)])[-1]
    FDR = -np.log10(FDR) * signs

    ## Save maps.
    for n, space in enumerate(spaces):
        out_dir = 'fmri_second_levels/FINAL.%s.%s.%s.%s' %(sm, fd, space, contrast)
        np.save(os.path.join(out_dir,'%s_%s_fdr' %(space,contrast)), FDR[indices==n])
        image = prepare_image(FDR[indices==n], space)
        image = nib.Nifti1Image(image, np.load(os.path.join(out_dir,'affine.npy')))
        nib.save(image, os.path.join(out_dir, 'fdr.nii.gz'))
    
print 'Done.'

# Section 5: Visualization

## Threshold Second-Level Maps
Thresholding clusters such that:
* p < 0.05 (FWE corrected, alpha = 0.05)
* Surface: clusters > 100mm2
* Volume: clusters > 20 contiguous voxels

In [ ]:
import os
import numpy as np
import nibabel as nib
from pandas import DataFrame
from scipy.sparse import coo_matrix
from mne.stats.cluster_level import _find_clusters as find_clusters
fs_dir = 'recons'

def load_sparse_coo(filename):
    npz = np.load(filename)
    M,N = npz['shape']
    return coo_matrix( (npz['data'], (npz['row'],npz['col'])), (M,N) )

def prepare_image(arr, space):
    npz = np.load('fmri_second_levels/%s_connectivity.npz' %space)
    image = np.zeros_like(npz['mapping'], dtype=float)
    
    if not space == 'mni305': 
        image[npz['vertices']] += arr
    else:
        x,y,z = npz['voxels'].T
        image[x,y,z] += arr
    
    for _ in range(4 - len(image.shape)): image = np.expand_dims(image,-1)
    return image

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## I/O parameters.
sm = 6
fd = 0.9
spaces = ['lh','rh','mni305']
contrasts = ['Delib','DelibMod','Antcp','AntcpMod','Shock']

## Thresholding parameters.
threshold = -np.log10( 0.05 )
min_cluster = dict(lh = 100, rh = 100, mni305 = 20)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

for space in spaces:
    
    print space,
    
    ## Load connectivity information.
    connectivity = load_sparse_coo('fmri_second_levels/%s_connectivity.npz' %space)

    ## Load mapping information.
    npz = np.load('fmri_second_levels/%s_connectivity.npz' %space)
    
    if not space == 'mni305':
        vertices = npz['vertices']
        average_area = nib.load(os.path.join(fs_dir, 'fsaverage', 'surf', '%s.white.avg.area.mgh' %space)).get_data()
        average_area = average_area[vertices].squeeze()
        
    for contrast in contrasts:
        
        ## Load FWE-corrected p-values.
        f = 'fmri_second_levels/FINAL2.%s.%s.%s.%s/%s_%s_fwe.npy' %(sm, fd, space, contrast, space, contrast)
        fwe = np.load(f)
        
        ## Find clusters.
        include = np.where(fwe,True,False)
        clusters, sums = find_clusters(fwe, threshold, tail=0, connectivity=connectivity, include=include, t_power=0)
        
        ## Compute areas.
        if not space == 'mni305': cluster_sums = np.array([average_area[c].sum() for c in clusters])
        else: cluster_sums = sums
            
        ## Threshold.
        try:
            survival_ix = np.concatenate([c for c, s in zip(clusters,cluster_sums) if s > min_cluster[space]])
            fwe[~np.in1d(np.arange(fwe.shape[0]), survival_ix)] = 0
        except ValueError:
            fwe = np.zeros_like(fwe)
        
        ## Save.
        image = prepare_image(fwe, space)
        image = nib.Nifti1Image(image, np.load(os.path.join(os.path.dirname(f),'affine.npy')))
        nib.save(image, os.path.join(os.path.dirname(f), 'fwe_thresh_%0.3f.nii.gz' %threshold))
        
print 'Done.'

## Compute Percent Signal Change

In [ ]:
import os
import numpy as np
import nibabel as nib
from pandas import read_csv
mri_dir = 'fmri_first_levels'

def prepare_image(arr, space):
    npz = np.load('fmri_second_levels/%s_connectivity.npz' %space)
    image = np.zeros_like(npz['mapping'], dtype=float)
    
    if not space == 'mni305': 
        image[npz['vertices']] += arr
    else:
        x,y,z = npz['voxels'].T
        image[x,y,z] += arr
    
    for _ in range(4 - len(image.shape)): image = np.expand_dims(image,-1)
    return image

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## I/O parameters.
sm = 6
fd = 0.9
spaces = ['lh','rh','mni305']
contrasts = ['Delib','DelibMod','Antcp','AntcpMod','Shock']
threshold = 1.301

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main Loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Get subjects list.
info = read_csv('demographics.csv')
subjects = info.loc[~info.Exlude, 'Subject'].as_matrix()

## Load average signal.
mean_signal = np.load('fmri_second_levels/mean_signal.npz')

for space in spaces:
    
    print space,
    
    ## Assemble design matrices.
    subj_dir = os.path.join(mri_dir, '%s', 'arc_001', 'FINAL2.%s.%s.%s' %(sm, fd, space), 'X.dat')
    scale_factors = np.array([np.loadtxt(subj_dir %subject).max(axis=0)[:len(contrasts)] 
                             for subject in subjects]).T

    for n, contrast in enumerate(contrasts):
        
        ## Load first levels.
        out_dir = 'fmri_second_levels/FINAL2.%s.%s.%s.%s' %(sm, fd, space, contrast)
        ces = np.load(os.path.join(out_dir, 'first_levels.npz'))['ces']
        
        ## Compute PSC (Pernet 2014, Frontiers in Neuroscience).
        ms = np.where(mean_signal[space], mean_signal[space], np.inf).T
        psc = np.divide(ces * scale_factors[n] * 100., ms)
        psc = prepare_image(psc.mean(axis=1), space)
        
        ## Mask image.
        fwe = nib.load(os.path.join(out_dir, 'fwe_thresh_%s.nii.gz' %threshold)).get_data()
        psc *= np.where(fwe,1,0)
        
        ## Save.
        image = nib.Nifti1Image(psc, np.load(os.path.join(out_dir,'affine.npy')))
        nib.save(image, os.path.join(out_dir, 'psc.nii.gz'))
        
print 'Done.'

## Surface Plots

In [ ]:
import os 
import numpy as np
from surfer import Brain
img_dir = 'plots/FINAL2/second_levels'
%matplotlib qt4

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

sm = 6
fd = 0.9
contrasts = ['Delib']
overlay = 'psc'
surface = 'inflated'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Plot.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

for hemi in ['lh','rh']: 

    for contrast in contrasts:

        fn = os.path.join('fmri_second_levels', 'FINAL.%s.%s.%s.%s' %(sm, fd, hemi, contrast), '%s.nii.gz' %overlay)
        
        for view in ['lateral','medial']:
            
            brain = Brain("fsaverage", hemi, surface)
            try: brain.add_overlay(fn, min=0.04, max=2.5, sign="pos")
            except: continue
            brain.show_view(view=view)
            od = os.path.join(img_dir, overlay, surface)
            if not os.path.isdir(od): os.makedirs(od)
            of = os.path.join(od, '%s_%s_%s.png' %(hemi,contrast,view))
            Brain.save_image(brain,of)

## Compute surface summary table

In [58]:
import os
import numpy as np
import nibabel as nib
from mne import Label, read_label, grow_labels, vertex_to_mni, set_log_level
set_log_level(verbose=False)
fs_dir = 'recons'
    
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## I/O parameters.
contrast = 'DelibMod'
thresh = 1.301
sm = 6
fd = 0.9

## ROI parameters.
extent = 10 #mm
grow = False
    
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

label_dir = 'fmri_second_levels/labels/seeds_%s' %contrast
labels = sorted([f for f in os.listdir(label_dir) if not f.startswith('fig') and f.endswith('label')])

fmni = open('%s/%s_surface_mni2.csv' %(label_dir,contrast), 'w')
fmni.write(','.join(['Label','V','X','Y','Z','PSC','F','p'])+'\n')

for f in labels:

    ## Extract info. Read label.
    roi, hemi = f.replace('.label', '').split('-')
    label = read_label(os.path.join(label_dir, f))
    
    ## Load accompanying overlay.
    f = 'fmri_second_levels/FINAL2.%s.%s.%s.%s/psc.nii.gz' %(sm,fd,hemi,contrast)
    overlay = nib.load(f).get_data().squeeze()

    ## Find maximum vertex.
    ix = np.argmax(overlay[label.vertices])
    v = label.vertices[ix]

    ## Extract MNI coordinates.
    x,y,z = vertex_to_mni(v, 0 if hemi=='lh' else 1, 'fsaverage', fs_dir)[0]
    
    ## Extract PSC, F-scores, p-values.
    f = 'fmri_second_levels/FINAL2.%s.%s.%s.%s/psc.nii.gz' %(sm,fd,hemi,contrast)
    psc = nib.load(f).get_data().squeeze()[v]

    f = 'fmri_second_levels/FINAL2.%s.%s.%s.%s/F.nii.gz' %(sm,fd,hemi,contrast)
    F = nib.load(f).get_data().squeeze()[v]
    
    f = 'fmri_second_levels/FINAL2.%s.%s.%s.%s/fwe_thresh_%s.nii.gz' %(sm,fd,hemi,contrast,thresh)
    p = nib.load(f).get_data().squeeze()[v]
    
    ## Write information.
    fmni.write('%s-%s,%s,%0.0f,%0.0f,%0.0f,%0.2f,%0.2f,%0.6f\n' %(roi,hemi,v,x,y,z,psc,F,10.**-p))
    
    if grow:
        
        ## Grow label.
        label = grow_labels('fsaverage', v, extent, 0 if hemi=='lh' else 1, subjects_dir=fs_dir,
                            names='fig_%s-%s' %(roi,hemi), surface='pial')[0]
        
        ## Ensure label is within actiation. Save.
        ix = np.in1d(label.vertices, np.where(overlay)[0])
        label.pos = label.pos[ix]
        label.values = label.values[ix]
        label.vertices = label.vertices[ix]
        label.save('%s/%s.label' %(label_dir, label.name))
    
fmni.close()
print 'Done.'

Done.


## Compute volume summary table

In [59]:
import os
import numpy as np
import nibabel as nib
from nibabel.affines import apply_affine
fs_dir = '/space/lilli/1/users/DARPA-Recons'
    
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## I/O parameters.
contrast = 'Delib'
sm = 6
fd = 0.9

## ROI parameters.
extent = 6 #mm

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
label_dir = 'fmri_second_levels/labels/seeds_%s' %contrast

## Initialize statistics file.
fmni = open('%s/%s_volume_mni2.csv' %(label_dir,contrast), 'w')
fmni.write(','.join(['Label','cX','cY','cZ','X','Y','Z','PSC','F','p'])+'\n')

## Load data. 
npz = np.load('fmri_second_levels/mni305_connectivity.npz')
affine = np.load('fmri_second_levels/FINAL2.%s.%s.mni305.%s/affine.npy' %(sm,fd,contrast))
obj = nib.load('fmri_second_levels/FINAL2.%s.%s.mni305.%s/psc.nii.gz' %(sm,fd,contrast))

overlay = obj.get_data().squeeze()
Fval = nib.load('fmri_second_levels/FINAL2.%s.%s.mni305.%s/F.nii.gz' %(sm,fd,contrast)).get_data().squeeze()
pval = nib.load('fmri_second_levels/FINAL2.%s.%s.mni305.%s/fwe_thresh_1.301.nii.gz' %(sm,fd,contrast)).get_data().squeeze()

rois = ['Left-Caudate', 'Left-Putamen', 'Left-Hippocampus',
        'Right-Caudate', 'Right-Putamen', 'Right-Hippocampus']
for roi in rois:
    
    ## Extract activated voxels in ROI.
    voxels = npz['voxels'][npz['names'] == roi]
    voxels = voxels[np.where(overlay[[arr for arr in voxels.T]])]
    
    ## Find maximally activated voxel.
    ix = np.argmax(overlay[[arr for arr in voxels.T]])
    center = voxels[ix]
    i,j,k = center
    
    ## Get MNI coordinates.
    x,y,z = apply_affine(affine, center)
    
    ## Extract max values.
    psc = overlay[i,j,k]
    F = Fval[i,j,k]
    p = pval[i,j,k]
    
    ## Write to file.
    fmni.write('%s,%0.0d,%0.0d,%0.0d,%0.0d,%0.2d,%0.2d,%0.2f,%0.2f,%0.6f\n' %(roi,i,j,k,x,y,z,psc,F,10.**-p))
    
    ## Create sphere: find all voxels within extent.
    dist = [np.linalg.norm( np.diff( apply_affine(affine,np.vstack([center,v])), axis=0 ) ) for v in voxels]
    ix = np.where(np.array(dist)<=extent)
    sphere = voxels[ix]
    
    ## Save.
    #hemi, roi = roi.split('-')
    #if hemi.startswith('L'): name = '%s-lh' %roi.lower()
    #else: name = '%s-rh' %roi.lower()
    #np.save(os.path.join(out_dir, name), sphere)
    
fmni.close()
print 'Done.'

Done.


## Post-hoc F-statistic Fix
Sam realized very late in the game he should have been saving out the pre-TFCE F-statistics. Fortunately these can be recomputed using the WLS code sans TFCE.

In [64]:
import os, sys
import numpy as np
from pandas import read_csv
from scipy.sparse import coo_matrix

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## I/O parameters.
sm = 6
fd = 0.9
space = 'lh'
contrast = 'DelibMod'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Load and prepare data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

def load_sparse_coo(filename):
    npz = np.load(filename)
    M,N = npz['shape']
    return coo_matrix( (npz['data'], (npz['row'],npz['col'])), (M,N) )

out_dir = os.path.join('fmri_second_levels', 'FINAL.%s.%s.%s.%s' %(sm,fd,space,contrast))

## Load data.
npz = np.load(os.path.join(out_dir, 'first_levels.npz'))
ces = npz['ces']
cesvar = np.abs( 1. / npz['cesvar'] )

## Define indices.
connectivity = load_sparse_coo(os.path.join('fmri_second_levels', '%s_connectivity.npz' %space))
index,  = np.where(~np.isinf(cesvar).sum(axis=1).astype(bool))
include = ~np.isinf(cesvar).sum(axis=1).astype(bool)
    
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Setup for permutation testing.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Load subject information.
info = read_csv(os.path.join('demographics.csv'))
info = info[~info.Exlude].reset_index()
n_subj, _ = info.shape

## Build Design Matrix.
X = np.zeros((n_subj,2))
X[:,0] = 1                                        # Intercept
X[:,1] = np.where(info.Scanner == 'Trio', 0, 1)   # Scanner
n_subj, n_pred = X.shape

sign_flips = np.ones((1,n_subj))
n_shuffles = sign_flips.shape[0]

## Preallocate arrays for results.
shape = [n_shuffles] + list(ces.shape[:-1])
Bmap = np.zeros(shape)
Fmap = np.zeros(shape)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

'''
Following the instructions of Winkler et al. (2014), we use the Freedman and Lane (1983)
permutation procedure. This allows us to precompute a number of values ahead of time.

To understand the WLS computations, please see:
https://github.com/statsmodels/statsmodels/blob/master/statsmodels/base/model.py
https://github.com/statsmodels/statsmodels/blob/master/statsmodels/regression/linear_model.py
'''

def wls(X,Y,W):
    B = np.linalg.inv(X.T.dot(W).dot(X)).dot(X.T).dot(W).dot(Y)
    ssr = W.dot( np.power(Y - np.dot(X,B),2) ).sum()
    scale = ssr / (n_subj - n_pred)
    cov_p = np.linalg.inv(X.T.dot(W).dot(X)) * scale
    F = np.power(B[0],2) * np.power(cov_p[0,0],-1)
    return B[0], F

## Loop it!
for n, sf in enumerate(sign_flips):
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Compute statistics.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    for m in index:

        ## Update variables.
        W = np.diag(cesvar[m])
        Y = ces[m]
        
        ## Permute values.
        ## See Winkler et al. (2014), pg. 385
        ## To compute Hat Matrix, see: https://en.wikipedia.org/wiki/Projection_matrix and 
        Z = X[:,1:]
        ZZ = Z.dot( np.linalg.inv( Z.T.dot(W).dot(Z) ) ).dot(Z.T).dot(W)
        Rz = np.identity(n_subj) - ZZ
        Y = np.diag(sf).dot(Rz).dot(Y)
        
        ## Perform WLS.
        Bmap[n,m], Fmap[n,m] = wls(X,Y,W) 

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Save results.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#  

def prepare_image(arr, space):
    npz = np.load('fmri_second_levels/%s_connectivity.npz' %space)
    image = np.zeros_like(npz['mapping'], dtype=float)
    
    if not space == 'mni305': 
        image[npz['vertices']] += arr
    else:
        x,y,z = npz['voxels'].T
        image[x,y,z] += arr
    
    for _ in range(4 - len(image.shape)): image = np.expand_dims(image,-1)
    return image

## Translate array back into proper space.
image = prepare_image(Fmap.squeeze(), space).squeeze()

## Load in results table.
if space == 'mni305':
    results = read_csv('fmri_second_levels/labels/seeds_%s/%s_volume_mni2.csv' %(contrast,contrast))
    fscores = [image[i,j,k] for i,j,k in results[['cX','cY','cZ']].as_matrix()]
    results['Fpre'] = fscores
    results.to_csv('fmri_second_levels/labels/seeds_%s/%s_volume_mni2.csv' %(contrast,contrast))
else:
    results = read_csv('fmri_second_levels/labels/seeds_%s/%s_surface_mni2.csv' %(contrast,contrast))
    if not 'Fpre' in results.columns: results['Fpre'] = np.nan
    vertices = results.loc[[True if label.endswith(space) else False for label in results.Label],'V'].as_matrix()
    for v in vertices: results.loc[results.V==v,'Fpre'] = image[v]
    results.to_csv('fmri_second_levels/labels/seeds_%s/%s_surface_mni2.csv' %(contrast,contrast), index=False)
    
print 'Done.'

/home/szoro/Documents/software/anaconda2.7/lib/python2.7/site-packages/ipykernel/__main__.py:30: RuntimeWarning: divide by zero encountered in divide


Done.


# Section 6: Connectivity Analysis
This was exploratory analysis and ultimately dropped from the manuscript. Code is left in to document its existence and as a template for future projects.

## Extract Timeseries

In [ ]:
import os
import numpy as np
import nibabel as nib
from mne import read_label
from pandas import read_csv
from scipy.signal import detrend
mri_dir = '/autofs/space/lilli_002/users/DARPA-ARC'
label_dir = 'fmri_second_levels/labels'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## I/O parameters.
results_file = 'arc_hierarchical_add_FINAL_regressors'
sm = 6
fd = 0.9

## MRI parameters.
n_acq = 977
tr = 1.75
n_extract = 8

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Prepare regressors.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Load trial information.
results_file = 'stan_results/%s.csv' %results_file
df = read_csv(results_file)
subjects = np.unique(df.Subject)

## Load DCT basis set.
dct = np.loadtxt('fmri_second_levels/dct_0.02.txt')

## Define TR onsets.
tr_onsets = np.insert( np.cumsum( np.ones(n_acq - 1) * tr ), 0, 0 )

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
ROIs = sorted([f for f in os.listdir(label_dir) if f.endswith('label') or f.endswith('npy')])
names = [roi for roi in ROIs if '-lh.label' in roi] +\
        [roi for roi in ROIs if '-rh.label' in roi] +\
        [roi for roi in ROIs if 'npy' in roi]
names = [n.split('.')[0] for n in names]

for subject in subjects:
    
    print subject,
    subj_dir = os.path.join(mri_dir, subject, 'arc_001', '001')
    
    ## Find onsets of trials relative to acquisitions.
    trial_onsets = df.loc[df.Subject==subject, 'RiskOnset'].as_matrix()
    onsets = np.digitize(trial_onsets, tr_onsets) - 1
    
    ## Load motion regressors / censors.
    mc = np.loadtxt(os.path.join(subj_dir, 'FINAL.mc.par'))
    try:
        censor = np.loadtxt(os.path.join(subj_dir, 'FINAL.censor.%s.par' %fd))
        exclude = np.in1d(tr_onsets, censor)
    except IOError:
        exclude = np.zeros_like(tr_onsets).astype(bool)
    
    ## Determine inclusion.
    include = np.zeros(n_acq)
    for onset in onsets: include[onset:(onset+n_extract)] += 1
    include *= np.invert(exclude)
    
    extractions = np.zeros((len(ROIs), include.astype(int).sum()))
    
    n = 0
    for space in ['lh','rh','mni305']:
        
        ## Load MRI data.
        if space == 'mni305': f = 'fmcpr.sm%s.%s.2mm.b0dc.nii.gz' %(sm,space)
        else: f = 'fmcpr.sm%s.fsaverage.%s.b0dc.nii.gz' %(sm,space)
        data = nib.load(os.path.join(subj_dir, f)).get_data().squeeze()
        
        ## Setup ROIs.
        if space == 'lh': 
            rois = [roi for roi in ROIs if roi.endswith('-lh.label')]
            indices = [read_label(os.path.join(label_dir,roi)).vertices for roi in rois]
        elif space == 'rh': 
            rois = [roi for roi in ROIs if roi.endswith('-rh.label')]
            indices = [read_label(os.path.join(label_dir,roi)).vertices for roi in rois]
        else: 
            rois = [roi for roi in ROIs if roi.endswith('npy')]
            indices = [np.load(os.path.join(label_dir, roi)) for roi in rois]
            indices = [[arr for arr in ix.T] for ix in indices]

        ## Iterate over ROIs.
        for roi, ix in zip(rois, indices):
            
            ## Extract timecourse.
            ltc = data[ix].mean(axis=0)
            ltc = ltc[np.where(include)]
            
            ## Detrend.
            ltc = detrend(ltc, type='linear')
            ltc = detrend(ltc, type='constant')
            
            ## Store.
            extractions[n] = ltc
            n += 1
        
    ## Save.
    f = 'fmri_second_levels/labels/extractions/%s_extractions' %subject
    np.savez_compressed(f, data=extractions, mc=mc[np.where(include)], 
                        dct=dct[np.where(include)], labels=np.array(names))

print 'Done.'

## Assemble Data

In [ ]:
import os
import numpy as np
extractions_dir = 'fmri_second_levels/labels/extractions/'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

n_regressors = []
files = sorted(os.listdir(extractions_dir))
for n, f in enumerate(files):
    
    ## Load data. Make subject index.
    npz = np.load('%s/%s' %(extractions_dir,f))
    n_acq, n_mc = npz['mc'].shape
    ix = np.ones(n_acq) * n
    
    ## Tack on rows to mc.
    mc = np.hstack([npz['mc'], np.zeros([n_acq,6-n_mc])])
    n_regressors.append(n_mc)
    
    ## Assemble data.
    if not n:
        data = npz['data'].T
        motion = mc.copy()
        dct = npz['dct']
        index = ix.copy()
    else:
        data = np.concatenate([data, npz['data'].T], axis=0)
        motion = np.concatenate([motion, mc.copy()], axis=0)
        dct = np.concatenate([dct, npz['dct']], axis=0)
        index = np.concatenate([index,ix])
        
## Finalize information.
subjects = np.array([f.split('_')[0] for f in files])
n_regressors = np.array(n_regressors)
index += 1

## Save.
np.savez_compressed('fmri_second_levels/labels/extractions', subjects=subjects, index=index, data=data, 
                    dct=dct, motion=motion, n_regressors=n_regressors, labels=npz['labels'])
print 'Done.'

## Partial Correlation

In [ ]:
import cPickle, os, pystan
import numpy as np
import pylab as plt
import seaborn as sns
from pandas import DataFrame
from itertools import combinations
sns.set_style('white')
%matplotlib inline

def zscore(arr): return (arr - arr.mean()) / arr.std()

def partial_corr(corr):
    prec = np.linalg.inv(corr)
    I,J = np.tril_indices_from(prec,k=-1)
    return np.array([ -prec[i,j] / np.sqrt(prec[i,i]*prec[j,j]) for i,j in zip(I,J) ])

def init(N,M):
    d = dict()
    d['L_Sigma_mu'] = np.linalg.cholesky( (np.ones((M,M)) + np.identity(M)) / 2. )
    d['L_Sigma'] = np.array([np.identity(M) for _ in range(N)])
    return d

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## ROI parameters (NOTE: no pre-motor).
labels = ['dacc-lh', 'dlpfc-lh', 'insula-lh', 'mcc-lh', 'orbitalis-lh', 'pre_sma-lh', 
          'dacc-rh', 'dlpfc-rh', 'insula-rh', 'mcc-rh', 'orbitalis-rh', 'pre_sma-rh',
          'caudate-lh', 'hippocampus-lh', 'hippocampus-rh', 'putamen-lh','putamen-rh']

## Stan parameters.
model_name = 'partial_corr_hlkj'

iter = 2000
chains = 4
thin = 4
n_jobs = 2

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Load and prepare data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Load data.
npz = np.load('fmri_second_levels/labels/extractions.npz')
data = npz['data']
index = npz['index']

## Limit to labels of interest.
labels = np.array(labels)
data = data[:,np.in1d(npz['labels'], labels)]

for n in np.unique(index):
    
    ## Assemble nuisance regressors.
    ix = np.where(index==n)[0]
    regressors = np.hstack([npz['dct'][ix], npz['motion'][ix]])
    regressors = regressors[:,np.where(regressors.sum(axis=0), True, False)]
    
    ## Perform nuisance regression.
    beta, _, _, _ = np.linalg.lstsq(regressors, data[ix])
    data[ix] -= np.dot(regressors,beta)
    
    ## Normalize (z-score).
    data[ix] = np.apply_along_axis(zscore, 0, data[ix])
    
## Final preparation.
_, nrow = np.unique(index, return_counts=True)
N, M, T = nrow.shape[0], labels.shape[0], nrow.max()

Y = np.zeros((N,T,M))
for n, r in enumerate(nrow): Y[n,:r,:] = data[index == n+1]

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Pystan.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Assemble data.
init = [init(N,M) for _ in range(chains)]
data = dict(N=N, M=M, T=T, Y=Y, nrow=nrow)

## Fit model.
print 'Running Stan.',
f = 'stan_models/%s.txt' %model_name
fit = pystan.stan(f, data=data, chains=chains, iter=iter, thin=thin, init=init, 
                  seed=47404, n_jobs=n_jobs)
print 'Finished.'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## Save summary file.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

summary = fit.summary()
summary = DataFrame(summary['summary'], columns=summary['summary_colnames'], index=summary['summary_rownames'])
f = 'stan_results/%s.csv' %model_name
summary.to_csv(f)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## Extract Results.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

results = fit.extract()

## Append data to results.
results['Subjects'] = npz['subjects']
results['Labels'] = labels

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## Save results.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
print 'Saving data.'

## Save all data.
f = 'stan_results/%s.pickle' %model_name
with open(f, 'wb') as f: cPickle.dump(results, f)

print 'Done.'

In [ ]:
## Extract group correlation matrix.
sigma_mu = results['Sigma_mu']

## Contrast partial correlation matrices.
# data = np.array([partial_corr(corr) for corr in sigma_mu[:,ix][:,:,ix]])
data = np.array([partial_corr(corr) for corr in sigma_mu])
pairs = np.array(list(combinations(labels,2)))

## Sort items
ix = np.argsort(np.median(data, axis=0))[::-1]
data = data[:,ix]
pairs = pairs[ix]

fig, ax = plt.subplots(1,1,figsize=(12,6))
sns.violinplot(data=data, cut=0, ax=ax)
ax.hlines(0, 0, len(pairs), color='c', alpha=0.4)